# Quantitative Volatility Platform - Demo Walkthrough

This notebook demonstrates the complete capabilities of the QVP platform, including:
- Data ingestion and volatility estimation
- Strategy backtesting with realistic transaction costs
- Performance analytics and visualization
- Risk management and portfolio optimization

## Setup

In [ ]:
import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.insert(0, '..')

from qvp.data.ingestion import DataIngester
from qvp.research.volatility import VolatilityEstimator
from qvp.research.garch import GARCHModeler, compare_garch_models
from qvp.research.features import TechnicalIndicators, VolatilityFeatures
from qvp.backtest.engine import BacktestEngine
from qvp.strategies.volatility_strategies import VIXMeanReversionStrategy, SimpleVolatilityStrategy
from qvp.analytics.performance import PerformanceMetrics, RollingMetrics

print("✓ All modules imported successfully")

## 1. Data Ingestion

Download market data for SPY and VIX indices

In [ ]:
# Initialize data ingester
ingester = DataIngester()

# Download data (cached after first run)
print("Downloading SPY data...")
spy_data = ingester.download_equity_data('SPY', start_date='2021-01-01', end_date='2024-12-31')

print("\nDownloading VIX data...")
vix_data = ingester.download_vix_data(start_date='2021-01-01', end_date='2024-12-31')

print(f"\n✓ Downloaded {len(spy_data)} days of SPY data")
print(f"✓ Downloaded {len(vix_data)} days of VIX data")

# Display summary statistics
print("\nSPY Summary Statistics:")
print(spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].describe())

## 2. Volatility Estimation

Calculate volatility using multiple estimators and compare efficiency

In [ ]:
# Calculate all volatility estimators
vol_estimator = VolatilityEstimator()
window = 20

vol_estimates = pd.DataFrame(index=spy_data.index)
vol_estimates['close_to_close'] = vol_estimator.close_to_close(spy_data, window=window)
vol_estimates['parkinson'] = vol_estimator.parkinson(spy_data, window=window)
vol_estimates['garman_klass'] = vol_estimator.garman_klass(spy_data, window=window)
vol_estimates['rogers_satchell'] = vol_estimator.rogers_satchell(spy_data, window=window)
vol_estimates['yang_zhang'] = vol_estimator.yang_zhang(spy_data, window=window)

# Annualize volatility (assume 252 trading days)
vol_estimates_annual = vol_estimates * np.sqrt(252)

print("Volatility Estimator Efficiency Comparison:")
print("Close-to-Close:  1.0x (baseline)")
print("Parkinson:       5.0x more efficient")
print("Garman-Klass:    7.7x more efficient")
print("Rogers-Satchell: 5.2x more efficient")
print("Yang-Zhang:     14.0x more efficient")

# Display recent estimates
print("\nRecent Volatility Estimates (Annualized):")
print(vol_estimates_annual.tail())

In [ ]:
# Visualize volatility estimates
fig = go.Figure()

for col in vol_estimates_annual.columns:
    fig.add_trace(go.Scatter(
        x=vol_estimates_annual.index,
        y=vol_estimates_annual[col],
        name=col.replace('_', ' ').title(),
        mode='lines'
    ))

fig.update_layout(
    title='SPY Volatility Estimates (20-day window, Annualized)',
    xaxis_title='Date',
    yaxis_title='Volatility',
    hovermode='x unified',
    height=500
)

fig.show()

## 3. GARCH Modeling

Fit GARCH models and compare their performance

In [ ]:
# Calculate returns
returns = spy_data['Close'].pct_change().dropna() * 100  # Convert to percentage

# Compare GARCH models
print("Fitting GARCH models...")
comparison = compare_garch_models(
    returns=returns,
    models=['GARCH', 'EGARCH', 'GJR-GARCH'],
    p=1,
    q=1
)

print("\nGARCH Model Comparison:")
print(comparison)

# Get best model
best_model = comparison.loc[comparison['aic'].idxmin(), 'model']
print(f"\n✓ Best model by AIC: {best_model}")

In [ ]:
# Fit best GARCH model and forecast
garch = GARCHModeler()
result = garch.fit(returns, model_type=best_model, p=1, q=1)

# Generate forecast
forecast = garch.forecast(horizon=30)

print(f"\n{best_model} 30-Day Volatility Forecast (Annualized):")
forecast_annual = np.sqrt(forecast * 252)
print(forecast_annual)

## 4. Technical Indicators & Features

Calculate technical indicators and volatility-specific features

In [ ]:
# Calculate technical indicators
indicators = TechnicalIndicators()

spy_data['rsi'] = indicators.rsi(spy_data, period=14)
macd_result = indicators.macd(spy_data)
spy_data['macd'] = macd_result['macd']
spy_data['macd_signal'] = macd_result['signal']
spy_data['macd_hist'] = macd_result['histogram']
spy_data['atr'] = indicators.atr(spy_data, period=14)

# Calculate volatility features
vol_features = VolatilityFeatures()
spy_data['realized_vol'] = vol_estimator.yang_zhang(spy_data, window=20) * np.sqrt(252)
vix_aligned = vix_data['Close'].reindex(spy_data.index, method='ffill')
spy_data['vol_spread'] = vol_features.volatility_spread(vix_aligned, spy_data['realized_vol'])
spy_data['vol_ratio'] = vol_features.volatility_ratio(vix_aligned, spy_data['realized_vol'])
spy_data['vol_zscore'] = vol_features.volatility_zscore(vix_aligned, window=60)

print("Technical Indicators & Volatility Features:")
print(spy_data[['Close', 'rsi', 'macd', 'atr', 'realized_vol', 'vol_spread', 'vol_zscore']].tail(10))

In [ ]:
# Visualize indicators
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('SPY Price', 'RSI', 'MACD', 'Volatility Spread'),
    row_heights=[0.4, 0.2, 0.2, 0.2]
)

# Price
fig.add_trace(go.Scatter(x=spy_data.index, y=spy_data['Close'], name='SPY', line=dict(color='blue')), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(x=spy_data.index, y=spy_data['rsi'], name='RSI', line=dict(color='purple')), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

# MACD
fig.add_trace(go.Scatter(x=spy_data.index, y=spy_data['macd'], name='MACD', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=spy_data.index, y=spy_data['macd_signal'], name='Signal', line=dict(color='orange')), row=3, col=1)

# Volatility Spread
fig.add_trace(go.Scatter(x=spy_data.index, y=spy_data['vol_spread'], name='Vol Spread', line=dict(color='red')), row=4, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=4, col=1)

fig.update_layout(height=1000, showlegend=True, hovermode='x unified')
fig.update_xaxes(title_text="Date", row=4, col=1)

fig.show()

## 5. Strategy Backtesting

Run backtests for VIX Mean Reversion and Simple Volatility Filter strategies

In [ ]:
# Load backtest results from demo script
vix_mr_equity = pd.read_csv('../data/results/vix_mean_reversion_equity.csv', index_col=0, parse_dates=True)
simple_equity = pd.read_csv('../data/results/simple_vol_filter_equity.csv', index_col=0, parse_dates=True)
vix_mr_metrics = pd.read_csv('../data/results/tearsheet_vix_mr.csv')
simple_metrics = pd.read_csv('../data/results/tearsheet_simple.csv')

print("✓ Loaded backtest results")
print(f"  VIX Mean Reversion: {len(vix_mr_equity)} days")
print(f"  Simple Vol Filter:  {len(simple_equity)} days")

In [ ]:
# Display performance metrics
print("\n" + "="*80)
print("STRATEGY PERFORMANCE COMPARISON")
print("="*80)

metrics_comparison = pd.merge(
    vix_mr_metrics[['Metric', 'Value']].rename(columns={'Value': 'VIX Mean Reversion'}),
    simple_metrics[['Metric', 'Value']].rename(columns={'Value': 'Simple Vol Filter'}),
    on='Metric'
)

# Format metrics for display
metrics_comparison['VIX Mean Reversion'] = metrics_comparison['VIX Mean Reversion'].astype(float)
metrics_comparison['Simple Vol Filter'] = metrics_comparison['Simple Vol Filter'].astype(float)

print(metrics_comparison.to_string(index=False))

# Highlight key metrics
print("\n" + "="*80)
print("KEY PERFORMANCE INDICATORS")
print("="*80)

key_metrics = ['total_return', 'sharpe_ratio', 'max_drawdown_pct', 'win_rate']
for metric in key_metrics:
    vix_val = metrics_comparison[metrics_comparison['Metric'] == metric]['VIX Mean Reversion'].values[0]
    simple_val = metrics_comparison[metrics_comparison['Metric'] == metric]['Simple Vol Filter'].values[0]
    
    if metric in ['total_return', 'sharpe_ratio', 'win_rate']:
        vix_fmt = f"{vix_val:.2%}" if metric != 'sharpe_ratio' else f"{vix_val:.3f}"
        simple_fmt = f"{simple_val:.2%}" if metric != 'sharpe_ratio' else f"{simple_val:.3f}"
    else:
        vix_fmt = f"{vix_val:.2%}"
        simple_fmt = f"{simple_val:.2%}"
    
    print(f"{metric.replace('_', ' ').title():20s}: VIX MR = {vix_fmt:>10s}  |  Simple Vol = {simple_fmt:>10s}")

In [ ]:
# Visualize equity curves
fig = go.Figure()

# VIX Mean Reversion
fig.add_trace(go.Scatter(
    x=vix_mr_equity.index,
    y=vix_mr_equity['equity'],
    name='VIX Mean Reversion',
    line=dict(color='blue', width=2)
))

# Simple Volatility Filter
fig.add_trace(go.Scatter(
    x=simple_equity.index,
    y=simple_equity['equity'],
    name='Simple Vol Filter',
    line=dict(color='green', width=2)
))

# Buy & Hold benchmark
benchmark = spy_data.loc[vix_mr_equity.index[0]:, 'Close']
benchmark_equity = (benchmark / benchmark.iloc[0]) * 1_000_000
fig.add_trace(go.Scatter(
    x=benchmark_equity.index,
    y=benchmark_equity.values,
    name='SPY Buy & Hold',
    line=dict(color='gray', width=2, dash='dash')
))

fig.update_layout(
    title='Strategy Equity Curves Comparison',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    hovermode='x unified',
    height=600,
    legend=dict(x=0.02, y=0.98)
)

fig.show()

In [ ]:
# Calculate and visualize drawdowns
def calculate_drawdown(equity_series):
    running_max = equity_series.expanding().max()
    drawdown = (equity_series - running_max) / running_max
    return drawdown

vix_mr_dd = calculate_drawdown(vix_mr_equity['equity'])
simple_dd = calculate_drawdown(simple_equity['equity'])
benchmark_dd = calculate_drawdown(benchmark_equity)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=vix_mr_dd.index,
    y=vix_mr_dd * 100,
    name='VIX Mean Reversion',
    fill='tozeroy',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=simple_dd.index,
    y=simple_dd * 100,
    name='Simple Vol Filter',
    fill='tozeroy',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=benchmark_dd.index,
    y=benchmark_dd * 100,
    name='SPY Buy & Hold',
    fill='tozeroy',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title='Drawdown Analysis',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    hovermode='x unified',
    height=500
)

fig.show()

## 6. Rolling Performance Metrics

Analyze time-varying performance characteristics

In [ ]:
# Calculate rolling metrics
rolling_metrics = RollingMetrics()

vix_mr_returns = vix_mr_equity['equity'].pct_change().fillna(0)
simple_returns = simple_equity['equity'].pct_change().fillna(0)

# Rolling Sharpe ratio (60-day window)
vix_mr_sharpe = rolling_metrics.rolling_sharpe(vix_mr_returns, window=60)
simple_sharpe = rolling_metrics.rolling_sharpe(simple_returns, window=60)

# Rolling volatility (60-day window)
vix_mr_vol = rolling_metrics.rolling_volatility(vix_mr_returns, window=60)
simple_vol = rolling_metrics.rolling_volatility(simple_returns, window=60)

print("✓ Calculated rolling performance metrics")

In [ ]:
# Visualize rolling metrics
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Rolling Sharpe Ratio (60-day)', 'Rolling Volatility (60-day)'),
)

# Rolling Sharpe
fig.add_trace(go.Scatter(
    x=vix_mr_sharpe.index,
    y=vix_mr_sharpe,
    name='VIX Mean Reversion',
    line=dict(color='blue')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=simple_sharpe.index,
    y=simple_sharpe,
    name='Simple Vol Filter',
    line=dict(color='green')
), row=1, col=1)

# Rolling Volatility
fig.add_trace(go.Scatter(
    x=vix_mr_vol.index,
    y=vix_mr_vol * 100,
    name='VIX Mean Reversion',
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=simple_vol.index,
    y=simple_vol * 100,
    name='Simple Vol Filter',
    line=dict(color='green'),
    showlegend=False
), row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=1)
fig.update_yaxes(title_text="Volatility (%)", row=2, col=1)

fig.update_layout(height=800, hovermode='x unified')
fig.show()

## 7. Returns Distribution Analysis

Analyze the statistical properties of strategy returns

In [ ]:
# Calculate returns statistics
from scipy import stats

def analyze_returns(returns, name):
    print(f"\n{name} Returns Analysis:")
    print("="*50)
    print(f"Mean Daily Return:    {returns.mean():.4%}")
    print(f"Median Daily Return:  {returns.median():.4%}")
    print(f"Std Dev:              {returns.std():.4%}")
    print(f"Skewness:             {stats.skew(returns):.4f}")
    print(f"Kurtosis:             {stats.kurtosis(returns):.4f}")
    print(f"Min Return:           {returns.min():.4%}")
    print(f"Max Return:           {returns.max():.4%}")
    
analyze_returns(vix_mr_returns, "VIX Mean Reversion")
analyze_returns(simple_returns, "Simple Vol Filter")

In [ ]:
# Visualize returns distribution
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('VIX Mean Reversion', 'Simple Vol Filter')
)

# VIX MR histogram
fig.add_trace(go.Histogram(
    x=vix_mr_returns * 100,
    name='VIX MR',
    nbinsx=50,
    marker_color='blue',
    opacity=0.7
), row=1, col=1)

# Simple Vol histogram
fig.add_trace(go.Histogram(
    x=simple_returns * 100,
    name='Simple Vol',
    nbinsx=50,
    marker_color='green',
    opacity=0.7
), row=1, col=2)

fig.update_xaxes(title_text="Daily Return (%)", row=1, col=1)
fig.update_xaxes(title_text="Daily Return (%)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=1)

fig.update_layout(
    title='Returns Distribution',
    showlegend=False,
    height=500
)

fig.show()

## 8. Summary & Conclusions

### Platform Capabilities Demonstrated:

1. **Data Infrastructure**
   - Automated data ingestion from Yahoo Finance
   - Caching system for efficient data access
   - Quality validation and cleaning

2. **Volatility Research**
   - 5 different volatility estimators (up to 14x more efficient than close-to-close)
   - GARCH family models (GARCH, EGARCH, GJR-GARCH)
   - Volatility forecasting capabilities

3. **Feature Engineering**
   - Technical indicators (RSI, MACD, Bollinger Bands, ATR)
   - Volatility-specific features (spread, ratio, z-score)
   - Regime detection algorithms

4. **Backtesting Framework**
   - Event-driven architecture
   - Realistic transaction costs (commission + slippage)
   - Portfolio tracking with P&L attribution

5. **Strategy Implementation**
   - VIX Mean Reversion (31.7% total return, 0.248 Sharpe)
   - Simple Volatility Filter (25.9% total return, 0.239 Sharpe)
   - Extensible strategy framework

6. **Performance Analytics**
   - Comprehensive metrics (Sharpe, Sortino, Calmar, etc.)
   - Rolling performance analysis
   - Drawdown monitoring

### Next Steps:

- Implement additional strategies (volatility risk premium, dispersion trading)
- Add options trading capabilities with Greeks calculations
- Build interactive dashboard for live monitoring
- Integrate machine learning models for regime detection
- Implement portfolio optimization with multiple strategies